## Parameter pair correlation

This notebook allows comparison of user-specified pairs of parameters from the algorithm,
supporting examination of parameter correlation.

### Running over Colab
Uncomment the commands in the following cell to install in Colab.
Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment and then proceed to the following cells.

In [ ]:
# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.9 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch main
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
import arviz as az
import numpy as np
from itertools import combinations
from emutools.utils import load_param_info
from inputs.constants import RUN_IDS, RUNS_PATH, PRIMARY_ANALYSIS, BURN_IN, OUTPUTS_PATH
from emutools.calibration import plot_3d_param_hist, plot_3d_param_corr

In [ ]:
idata = az.from_netcdf(RUNS_PATH / RUN_IDS[PRIMARY_ANALYSIS] / 'output/calib_full_out.nc')
idata = idata.sel(draw=np.s_[BURN_IN:])

Available parameters for comparison are:

In [ ]:
list(idata.posterior.keys())

In [ ]:
key_params = [
    'contact_rate',
    'start_cdr',
    'infectious_period',
    'imm_prop',
    'natural_immunity_period',
]

In [ ]:
for params in combinations(key_params, 3):
    fig = plot_3d_param_corr(idata, params, marker_size=3, n_samples=5000)
    fig.write_html(OUTPUTS_PATH / f'corr_{params[0]}X{params[1]}X{params[2]}.html')

In [ ]:
# abbreviations = load_param_info()['abbreviations']
# for params in combinations(key_params, 3):
#     print(f'- [{abbreviations[params[0]]} versus {abbreviations[params[1]]} versus {abbreviations[params[2]]}](corr_{params[0]}X{params[1]}X{params[2]})')